<a href="https://colab.research.google.com/github/abhishektripathi66/RecomendationSystem/blob/main/nextWordRecomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random


In [3]:
import pickle

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [6]:
from nltk.tokenize import RegexpTokenizer

In [7]:
from tensorflow.keras.models import Sequential, load_model

In [8]:
from tensorflow.keras.layers import LSTM, Dense, Activation

In [9]:
from tensorflow.keras.optimizers import RMSprop

In [10]:
text_df = pd.read_csv("https://raw.githubusercontent.com/lutzhamel/fake-news/master/data/fake_or_real_news.csv")

In [11]:
# joining all the texts from the csv
text= list(text_df.text.values)
joined_text = " ".join(text)

In [12]:
# due to the RAM restriction using only partial word
partial_text = joined_text[:100000]

In [13]:
# tokenizing only words and not space and special character
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [14]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}


In [15]:
# words used for making the prediction of next word, keeping it as 1 wont make much sense as it wont give any context
n_words = 10
input_words = []
next_words = []
for i in range(len(tokens)-n_words):
  input_words.append(tokens[i:i+n_words])
  next_words.append(tokens[i+n_words])

In [16]:
x = np.zeros((len(input_words), n_words,len(unique_tokens)),dtype=bool)
y= np.zeros((len(next_words),len(unique_tokens)),dtype=bool)

In [17]:
for i, words in enumerate(input_words):
  for j, word in enumerate(words):
    x[i,j, unique_token_index[word]] = 1
  y[i, unique_token_index[next_words[i]]] = 1

In [18]:
model = Sequential()
model.add(LSTM(128,input_shape=(n_words,len(unique_tokens)),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer= RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(x,y,batch_size=128,epochs=30, shuffle=True)

In [20]:
model.save("mymodel.keras")

In [21]:
model = load_model("mymodel.keras")

In [22]:
def predict_next_word(input_text,n_best):
  input_text=input_text.lower()
  x= np.zeros((1,n_words,len(unique_tokens)))
  for i,word in enumerate(input_text.split()):
    x[0,i,unique_token_index[word]]=1

  predictions=model.predict(x)[0]
  return np.argpartition(predictions,-n_best)[-n_best:]



In [ ]:
possible = predict_next_word("He will have to look into this thing and he",5)

In [24]:
print([unique_tokens[idx] for idx in possible])

['began', 'manages', 'were', 'will', 'pointing']


In [25]:
def generate_text(input_text,text_length,creativity=3):
  word_sequence = input_text.split()
  current=0
  for _ in range(text_length):
    sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
    try:
      choice=unique_tokens[random.choice(predict_next_word(sub_sequence,creativity))]
    except:
      choice = random.choice(unique_tokens)
    word_sequence.append(choice)
    current+=1
  return " ".join(word_sequence)

In [ ]:
generate_text("He will have to look into this thing and he",100,5)